In [1]:
#pytorch utility imports
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid
import cv2
#neural net imports
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np

In [14]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import json

In [2]:
def read_data(num, num2='0'):
    f = open('train/train_' + num + '_digits_'+ num2 + '.json')

    data = json.load(f)

    f.close()
    data = np.asarray(data)
  
    return data

def create_dataset():
  res = np.array([])
  target = np.array([[]])
  for num in range(9):
    img = cv2.imread('train/train_' + str(num) + '.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    src, _ = detect(img)
    for i,v in enumerate(src):
        data = read_data(str(num), str(i))
        target = np.concatenate((target, data.reshape(-1, 81)), axis=None)

        w = masked(img, v)
        tmp = np.asarray(list(gen(w)))
        if len(res) == 0:
            res = tmp.reshape(len(tmp), -1)
        else:
            res = np.vstack((res, tmp.reshape(len(tmp), -1)))
    
    
  return res, target

In [1]:
import numpy as np
import cv2 as cv
import operator
from skimage.segmentation import clear_border
import torch
import math
from imutils.perspective import four_point_transform
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
'''We are getting numbers in this chank of code'''


def mask_prediction(image):
    imgRGB = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    imgGREY = cv.cvtColor(imgRGB, cv.COLOR_RGB2GRAY)
    
    #4 th seminar has a great potential to make a nice mask to distinguish sudoku
    HLS = cv.cvtColor(imgRGB, cv.COLOR_RGB2HLS)
    LIGHT = HLS[:, :, 1]
    maskHLS = (LIGHT < 100)
    maskINT = maskHLS.astype(np.uint8)
    contours, useless_param = cv.findContours(maskINT, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE) 
    
    contour_areas = [cv.contourArea(c) for c in contours] 
    
    sort_areas_inds = sorted(range(len(contour_areas)), key = lambda k: contour_areas[k], reverse=True) 
    largest_contours=sort_areas_inds[:10]

    
    sudokus_contour=[]
    zero_mask = np.zeros((imgGREY.shape[:2]), np.uint8)
    for i in range(len(largest_contours)):
        if (contour_areas[largest_contours[i]] > 0.5 * contour_areas[largest_contours[0]]) and (contour_areas[largest_contours[i]] > 550000) : #the second condition will find second sudoku if any
            cv.drawContours(zero_mask, [contours[largest_contours[i]]], 0, (255,0,0), -2) #adding largest contours
            sudokus_contour.append(contours[largest_contours[i]])
    
    mask = np.bool_(zero_mask)
    return mask


def preProcess(img):
    height, width = 3000, 3000
    img = cv.resize(img, (width,height))
    imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    imgBlur = cv.GaussianBlur(imgGray, (5,5), 1)
    imgThreshold = cv.adaptiveThreshold(imgBlur, 255, 1, 1, 11, 2)
    contours, hieracrchy = cv.findContours(imgThreshold, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    # imgThreshold = cv.cvtColor(imgThreshold, cv.COLOR_BGR2RGB)
    return imgThreshold, contours

def reorder(myPoints):
    myPoints = myPoints.reshape((4, 2))
    myPointsNew = np.zeros((4, 1, 2), dtype = np.int32)
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis = 1)
    myPointsNew[1] = myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew

def biggestContour(img, contours):
    biggest = np.array([])
    height, width = 3000, 3000
    img = cv.resize(img, (width,height))
    max_area = 0
    for i in contours:
        area = cv.contourArea(i)
        if area > 50:
            peri = cv.arcLength(i, True)
            approx = cv.approxPolyDP(i, 0.02*peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    if biggest.size != 0:
            biggest = reorder(biggest)
            pts1 = np.float32(biggest)
            pts2 = np.float32([[0, 0], [img.shape[1], 0], [0, img.shape[1]], [img.shape[1], img.shape[0]]])
            matrix = cv.getPerspectiveTransform(pts1, pts2)
            imgWarpColored = cv.warpPerspective(img, matrix, (img.shape[1], img.shape[0]))
            imgBlank = np.zeros((450, 450, 3), np.uint8)
            imgWarpColored = cv.cvtColor(imgWarpColored, cv.COLOR_RGB2GRAY)
    return biggest, max_area, imgWarpColored

def splitBoxes_1 (img):
    img = cv.resize(img, (450,450))
    rows = np.vsplit(img, 9)
    boxes = []
    for r in rows:
        cols = np.hsplit(r, 9)
        for box in cols:
            boxes.append(box)
    return boxes

def get_cropped(sudokus_cont, image_grey): #returns arrray of sudoku fields from image
    sudokus=[] #list of images corresponding to cropped sudokus
    for i in sudokus_cont:
        #print(i)
        epsilon = 0.1* cv.arcLength(i, True)
        approx = cv.approxPolyDP(i, epsilon, True)
        #print(approx)
        #print(np.ravel(approx).shape)
        #print(type(approx))
        cropped_sudoku= four_point_transform(image_grey, np.ravel(approx).reshape(4,2))
        sudokus.append(cropped_sudoku)
    return sudokus


def splitBoxes_2 (img):
    for pieces in img:
        pieces = cv.resize(pieces, (450,450))
        rows = np.vsplit(pieces, 9)
        boxes = []
        for r in rows:
            cols = np.hsplit(r, 9)
            for box in cols:
                boxes.append(box)
    return boxes


def intializePredictionModel():
    model = load_model('myModel.h5')
    return model


def getPrediction (boxes):
    model = intializePredictionModel()
    result = []
    for image in boxes:
        #Image preparation
        img = np.asarray(image)
        img = img[4:img.shape[0] - 4, 4:img.shape[1] - 4]
        img = cv.resize(img, (28,28))
        img = img / 255
        img = img.reshape(1, 28, 28, 1)
        #Predict
        predictions = model.predict(img)
        #classIndex = model.predict_classes(img)
        classIndex = np.argmax(predictions, axis = -1)
        probabilityValue = np.amax(predictions)
        #print(classIndex, probabilityValue)
        #Saving
        if probabilityValue > 0.8:
            result.append(classIndex[0])
        else:
            result.append(-1)
    boards = np.array(result)
    boards = np.reshape(boards, (9,9))
    boards = [np.int16(boards)]
    return boards, result

def predict_image(img):
    mask = mask_prediction(img)
    imgThreshold, contours = preProcess(img)
    biggest, max_area, imgWarpColored = biggestContour(img, contours)
    boxes = splitBoxes_1 (imgWarpColored)
    # sudokus = get_cropped(sudokus_cont, img_grey)
    # boxes = splitBoxes_2 (sudokus)
    digits, result = getPrediction (boxes)
    return mask, digits

